# N-qubit teleportation with adaptive feed-forward noise

Generalises the 3-qubit feed-forward teleportation notebook to any number of qubits `N ≥ 3`. Qubits `0` and `1` start in a Bell pair, the rest are prepared with `X` gates, a CZ chain links neighbours from qubit `1` onward, intermediate qubits are measured in the X-basis, and the final qubit receives the sequence of feed-forward corrections `(X^{s_k}) H` described in the spec. A configurable depolarising + readout noise model matches the previous notebook.


> **Assumption:** All qubits except the final target are measured in the X-basis (qubits `1..N-2`). If you need to follow a different pattern—for example, measuring only a subset—pass an explicit `measure_qubits` list to `build_n_qubit_circuit`.


In [1]:
from __future__ import annotations

import math
from dataclasses import dataclass
from typing import List, Sequence

from qiskit import ClassicalRegister, QuantumCircuit, transpile
from qiskit.quantum_info import DensityMatrix, partial_trace, state_fidelity
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel, ReadoutError, depolarizing_error


In [2]:
@dataclass
class NoiseParams:
    p_single: float = 2e-3
    p_two: float = 1e-2
    p_readout: float = 1.5e-2


def build_noise_model(params: NoiseParams) -> NoiseModel:
    """Depolarising gate errors + readout flips, identical to the 3-qubit notebook."""
    noise_model = NoiseModel()

    single_error = depolarizing_error(params.p_single, 1)
    two_error = depolarizing_error(params.p_two, 2)
    readout_error = ReadoutError([[1 - params.p_readout, params.p_readout], [params.p_readout, 1 - params.p_readout]])

    single_gate_set = ["id", "x", "sx", "rz", "h"]
    two_gate_set = ["cx", "cz"]

    noise_model.add_all_qubit_quantum_error(single_error, single_gate_set)
    noise_model.add_all_qubit_quantum_error(two_error, two_gate_set)
    noise_model.add_all_qubit_readout_error(readout_error)
    return noise_model


In [3]:
def bell_pair_density(simulator: AerSimulator) -> DensityMatrix:
    circ = QuantumCircuit(2)
    circ.h(0)
    circ.cx(0, 1)
    circ.save_density_matrix(label="rho_orig")
    compiled = transpile(circ, simulator)
    result = simulator.run(compiled).result()
    return DensityMatrix(result.data(0)["rho_orig"])


In [4]:
def build_n_qubit_circuit(num_qubits: int, measure_qubits: Sequence[int] | None = None) -> QuantumCircuit:
    """Return the N-qubit teleportation circuit with programmable measurement set.

    Assumes qubits are labelled 0..N-1, we entangle (0,1), set qubits >=2 with X, link a CZ
    chain from 1 onwards, measure the requested qubits in the X basis, and for each measurement
    result apply the sequence (conditional X) followed by H on the final qubit as requested.
    """
    if num_qubits < 3:
        raise ValueError("Need at least 3 qubits for teleportation")

    if measure_qubits is None:
        measure_qubits = tuple(range(1, num_qubits - 1))  # skip final target

    measure_qubits = tuple(measure_qubits)
    num_clbits = len(measure_qubits)

    circ = QuantumCircuit(num_qubits, num_clbits)

    # 1. Entangle qubits (0,1)
    circ.h(0)
    circ.cx(0, 1)

    # 2. Prepare extra qubits with X (per the user spec)
    for q in range(2, num_qubits):
        circ.x(q)

    # 3. CZ chain from qubit 1 to the final target
    for q in range(1, num_qubits - 1):
        circ.cz(q, q + 1)

    # 4. Measure selected qubits in the X basis (H + Z measurement)
    for idx, q in enumerate(measure_qubits):
        circ.h(q)
        circ.measure(q, idx)

    target = num_qubits - 1

    # 5. Feed-forward sequence (X^{s_k}) H on the target, iterating from last measurement inward
    clbits = [circ.clbits[idx] for idx in range(num_clbits)]
    for idx in reversed(range(num_clbits)):
        with circ.if_test((clbits[idx], 1)):
            circ.x(target)
        circ.h(target)

    circ.save_density_matrix(label="rho_final")
    return circ


In [5]:
def reduced_pair(dm: DensityMatrix, keep: Sequence[int]) -> DensityMatrix:
    """Partial trace helper keeping only the requested qubits."""
    keep = sorted(keep)
    num_qubits = dm.num_qubits
    trace_out = [i for i in range(num_qubits) if i not in keep]
    return partial_trace(dm, trace_out)


def simulate_chain(num_qubits: int, simulator: AerSimulator) -> DensityMatrix:
    circuit = build_n_qubit_circuit(num_qubits)
    compiled = transpile(circuit, simulator)
    result = simulator.run(compiled).result()
    rho_full = DensityMatrix(result.data(0)["rho_final"])
    return reduced_pair(rho_full, keep=(0, num_qubits - 1))


In [6]:
def evaluate_fidelity(num_qubits: int, noise: NoiseParams) -> None:
    """Run both ideal and noisy simulators and print fidelities."""
    ideal_sim = AerSimulator(method="density_matrix")
    noisy_sim = AerSimulator(method="density_matrix", noise_model=build_noise_model(noise))

    rho_ref = bell_pair_density(ideal_sim)
    rho_ideal = simulate_chain(num_qubits, ideal_sim)
    rho_noisy = simulate_chain(num_qubits, noisy_sim)

    print(f"N = {num_qubits}")
    print(noise)
    print("Fidelity (ideal vs reference):", state_fidelity(rho_ref, rho_ideal))
    print("Fidelity (noisy vs reference):", state_fidelity(rho_ref, rho_noisy))


In [7]:
num_qubits = 5  # change this to explore larger chains (must be >= 3)
noise = NoiseParams(p_single=2e-3, p_two=1e-2, p_readout=1.5e-2)

evaluate_fidelity(num_qubits, noise)


N = 5
NoiseParams(p_single=0.002, p_two=0.01, p_readout=0.015)
Fidelity (ideal vs reference): 0.23144531250000036
Fidelity (noisy vs reference): 0.23672206403835386


In [8]:
print("\nChain length sensitivity (fixed noise):")
noise_model = build_noise_model(noise)
for n in range(3, 8):
    ideal_sim = AerSimulator(method="density_matrix")
    noisy_sim = AerSimulator(method="density_matrix", noise_model=noise_model)
    rho_ref = bell_pair_density(ideal_sim)
    rho_noisy = simulate_chain(n, noisy_sim)
    fidelity = state_fidelity(rho_ref, rho_noisy)
    print(f"  N={n}: fidelity={fidelity:.6f}")



Chain length sensitivity (fixed noise):
  N=3: fidelity=0.485236
  N=4: fidelity=0.250000
  N=5: fidelity=0.247889
  N=6: fidelity=0.250000
  N=7: fidelity=0.251831
